In [29]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from tqdm import tqdm

import base64
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import cv2

import pandas as pd

from glob import glob
import pickle

In [2]:
model = ResNet50(weights='imagenet', include_top=False)

avg_model = Sequential()
avg_model.add(model)
avg_model.add(GlobalAveragePooling2D())

def get_activations(pic_path):
    img = image.load_img(pic_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    
    return avg_model.predict(img_data)[0]

/anaconda3/envs/vkhack/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [50]:
import cv2

IMAGES_PATH = '/Users/vladgl/Downloads/vkhack/meme_pages_pics_old copy/'
images = sorted(glob(IMAGES_PATH + '*'))

def get_meme_pics(path):
    pics = glob(path + '/*')
    return pics
    
    result = []
    for pic in pics:
        img = cv2.imread(pic)
        img = cv2.resize(pic, (224, 224))
        result.append(img)
        
    return result

pic_paths = []
for i, image in enumerate(images):
    pic_paths += get_meme_pics(image)

In [53]:
len(pic_paths)

13484

In [55]:
len(activations_x)

13531

In [10]:
ACTIVATIONS_PATH = '/Users/vladgl/Downloads/vkhack/pic_activations/'
activations_lst = sorted(glob(ACTIVATIONS_PATH + '*'))

xs = []
ys = []

inverse_dict = dict()

pics_all = []

for i, activation in enumerate(activations_lst):
    inverse_dict[i] = activation
    activations = np.load(activation)
    activations_label = [i] * len(activations)
    
    xs.append(activations)
    ys.append(activations_label)

In [3]:
def load_meme_meta(path):
    pd_data = pd.read_csv(path, sep='\|\|\|', header=None, engine='python')
    
    return pd_data

def load_all_pics(path):
    pics = glob(path + '*')
    
    results = dict()
    for i in pics:
        pic = base64.b64encode(open(i, "rb").read()).decode('utf-8')
        results[i.replace('|', '/')] = pic
        
    return results

In [24]:
activations_x = np.load('../dump/all_activations.npy')
activations_y = np.load('../dump/all_meme_ys.npy')

with open('../dump/meme_map.pkl', 'rb') as f:
    inverse_map = pickle.load(f)
    
meme_meta = load_meme_meta('../dump/megaresult.txt')
pics = load_all_pics('../dump/images_main/')

In [37]:
def get_nearest_n(path, n):
    activations = get_activations(path)
    
    dists = np.linalg.norm(activations_x - activations, axis=1)
    dists_topn = np.argsort(dists)[:n]
    
    ys = activations_y[dists_topn]
    ys = ['meme_pages/' + inverse_map[y].split('/')[-1][:-8] for y in ys]
    
    vals, counts = np.unique(ys, return_counts=True)
    counts_sorted = np.argsort(counts)[::-1]
    return dists_topn

In [38]:
def get_neighbors_meta(nearest_n, counts):
    result = []
    
    data = meme_meta.loc[meme_meta[0].isin(nearest_n[:1])]
    result.append({'title': data[3].values[0],
                   'description': data[2].values[0],
                   'count': counts[0],
                   'link': data[5].values[0]})
    
    try:
        data = meme_meta.loc[meme_meta[0].isin(nearest_n[1:2])]
        result.append({'title': data[3].values[0],
                       'description': data[2].values[0],
                       'count': counts[1],
                       'link': data[5].values[0]})
    
        data = meme_meta.loc[meme_meta[0].isin(nearest_n[2:3])]
        result.append({'title': data[3].values[0],
                       'description': data[2].values[0],
                       'count': counts[2],
                       'link': data[5].values[0]})
    except:
        pass
    
    return result

In [ ]:
pic_paths[]

In [49]:
dists

array([ 7008,  7013,  7012,  7009,  6997,  6998,  7005,  7000,  7004,
        7007,  6999,  7002,  7011,  7010,  7006,  7003,  7001,   448,
        6996,  2894,   447,  6028,   449,  8570,  3896,  7861, 13118,
        8572,  8230,  7952,   734,  3478,   438,  2837,  1344,  1323,
        4172,  4761, 11350,   737,  6116, 11728,   741,  8158,  3279,
        4243,  1854,   451,  4237,  7621,  7954,   442, 11243,   443,
         238,  8231,  4762,  2884,  4242, 12623,  5605, 11136, 10868,
        7932, 10365,  4411,  4420,  4407, 13512,  6783,  3075, 11234,
        9188,  3774, 13119,  4247, 10829,  7288,  9014,  3443,  7933,
        7987,  1698, 11417,  4985,  5860,  5497,  5601,  3074,  2920,
        5496, 12252, 11133, 12613,  4417, 12879, 12615,  1695, 12258,
        7571, 12180,  7521,  4828,  5125,  6112,  6810,  5438,  8162,
       12115, 12362,  3295,  7931,  7567, 10270, 12612,  1188,  4427,
        1424, 12103, 11733, 12247,  2499,   735, 10668,  8381,  8278,
         450,  5995,

In [46]:
path = '../trash/pica.jpg'

In [47]:
dists = get_nearest_n(path, 256)

In [48]:
f = open('pica.txt', 'w')
for dist in dists:
    f.write(pic_paths[dist] + '\n')
f.close()